In [1]:
trace_directory = '~/.ros/tracing/profile'

In [2]:
import sys
# Assuming a workspace with:
#   src/tracetools_analysis/
#   src/micro-ROS/ros_tracing/ros2_tracing/tracetools_read/
sys.path.insert(0, '../')
sys.path.insert(0, '../../../micro-ROS/ros_tracing/ros2_tracing/tracetools_read/')
from collections import defaultdict
import datetime as dt
import os

from bokeh.palettes import Category20
from bokeh.plotting import figure
from bokeh.plotting import output_notebook
from bokeh.io import show
from bokeh.layouts import row
from bokeh.models import ColumnDataSource
from bokeh.models import DatetimeTickFormatter
from bokeh.models import PrintfTickFormatter
import numpy as np
import pandas as pd

from tracetools_analysis import utils
from tracetools_analysis.conversion import ctf
from tracetools_analysis.loading import load_pickle
from tracetools_analysis.processor import Processor
from tracetools_analysis.processor.ros2 import Ros2Handler
from tracetools_analysis.processor.profile import ProfileHandler

In [3]:
# Convert
trace_directory = os.path.expanduser(trace_directory)
pickle_filename = 'pickle'
pickle_path = os.path.join(trace_directory, pickle_filename)
count = ctf.convert(trace_directory, pickle_path)
print(f'{count} events')

47669 events


In [4]:
# Process
events = load_pickle(pickle_path)
address_to_func = {
    '0x7F8DBC0132F4': 'get_next_ready_executable',
    '0x7F8DBC012194': 'wait_for_work',
    '0x7F8DBC033638': 'collect_entities',
    '0x7F8DBC0347CC': 'add_handles_to_wait_set',
    '0x7F8DBBA5BFA3': 'rcl_wait',
    '0x7F8DBC0355B6': 'remove_null_handles',
}
profile_handler = ProfileHandler(address_to_func=address_to_func)
ros_handler = Ros2Handler()
processor = Processor(profile_handler, ros_handler)
processor.process(events)

In [9]:
data_util = utils.ProfileDataModelUtil(profile_handler.data)
ros_data_util = utils.RosDataModelUtil(ros_handler.data)

output_notebook()
psize = 450

# Plot durations
for tid in list(data_util.get_tids()):
    node_names = ros_data_util.get_node_names_from_tid(tid)
    node_names_format = ', '.join(node_names) if node_names is not None else 'unknown'
    functions_data = data_util.get_function_duration_data(tid)
    for function_data in functions_data:
    #if False:
        depth = function_data['depth']
        name = function_data['function_name']
        parent = function_data['parent_name']
        df = function_data['data']
        df = utils.DataModelUtil.convert_time_columns(
            df,
            ['duration_difference', 'actual_duration'],
            ['start_timestamp'],
        )

        # Duration vs. actual duration
        starttime = df['start_timestamp'].iloc[0].strftime('%Y-%m-%d %H:%M')
        duration_source = ColumnDataSource(df)
        duration = figure(
            title=f'Function duration over time: {name}()',
            x_axis_label=f'start ({starttime})',
            y_axis_label='duration (ms)',
            plot_width=psize, plot_height=psize,
        )
        duration.title.align = 'center'
        duration.varea_stack(
            ['actual_duration', 'duration_difference'],
            x='start_timestamp',
            color=('blue', 'red'),
            legend=('actual', 'wait'),
            source=duration_source,
        )
        duration.legend.label_text_font_size = '11px'
        duration.xaxis[0].formatter = DatetimeTickFormatter(seconds=['%Ss'])

        show(duration)
    
    # Comparison
    colors = Category20[20]

    data = {
        'depth': ['1'],
    }
    elements = []
    legend = []
    for function_data in functions_data:
        depth = function_data['depth']
        if depth != 1:
            continue
        name = function_data['function_name']
        actual_duration_mean = function_data['data']['actual_duration'].mean()
        duration_mean_diff = function_data['data']['duration_difference'].mean()
        actual_string = f'{name}, actual'
        diff_string = f'{name}, duration-actual'
        data[actual_string] = [actual_duration_mean]
        data[diff_string] = [duration_mean_diff]
        elements.append(actual_string)
        elements.append(diff_string)
        legend.append(f'{name}() actual: {actual_duration_mean:.4f}')
        legend.append(f'{name}() other: {duration_mean_diff:.4f}')

    comparison = figure(
        y_range=['1'],
        title=f'Mean durations comparison',
        x_axis_label='mean duration (ms)',
        y_axis_label='depth',
        #tools='hover',
        tooltips='$name: @$name{1.11}',
        plot_width=psize+200, plot_height=psize,
    )
    comparison.hbar_stack(
        elements, y='depth', height=0.3,
        color=colors[:(len(elements))],
        source=data,
        legend=legend,
    )
    comparison.legend.label_text_font_size = '10px'
    show(comparison)

Loading BokehJS ...